In [1]:
# import libraries
import time
import numpy as np
from keras.utils import np_utils
from keras.datasets import mnist
from numpy.core.fromnumeric import argmax

In [2]:
# load data 
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalization
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32) 
x_train = x_train / 255.0 
x_test = x_test / 255.0

x_train = x_train.reshape(-1, 28**2)
x_test = x_test.reshape(-1, 28**2)

# One hot encoding
y_train = np_utils.to_categorical(y_train,10).astype(np.float32)
y_test = np_utils.to_categorical(y_test,10).astype(np.float32)

# Number of unique classes
number_classes = y_test.shape[1]
print('Number of classes:',number_classes)

Number of classes: 10


In [3]:
# Activation function
def sigmoid(x):
  return 1. / (1. + np.exp(-x))

In [4]:
# IELM model
def IELM_Train(input_nodes, hidden_nodes, output_nodes,x,y):

  # Time
  start_time = time.time()

  a = np.empty((hidden_nodes,input_nodes))
  bias = np.empty((hidden_nodes,len(x)))
  H = np.empty((hidden_nodes,len(x)))
  beta = np.empty((hidden_nodes,output_nodes))
  d = y
  
  for i in range(0,hidden_nodes):
    
    a[i:i+1,:] = np.random.uniform(-1.,1.,size=(1,input_nodes))

    bias[i:i+1,:] = np.random.rand(1,len(x))

    p = np.dot(a[i:i+1,:],x.T) + bias[i:i+1,:]

    H[i:i+1,:] = sigmoid(p)

    H_inv = np.linalg.pinv(H[i:i+1,:])

    beta[i:i+1,:] = np.dot(H_inv.T,d)    

    d = d - np.dot(H[i:i+1,:].T,beta[i:i+1,:])

  y_pred = np.dot(H.T,beta) 

  end_time = time.time()

  Total_time = end_time - start_time 

  # Training Error
  error_training = 0

  for i in range(0,len(y)):
    desired_output = argmax(y_pred[i,:])
    actual_output = argmax(y[i,:]) 

    if actual_output!=desired_output:
      error_training+=1

  # Training Accuracy 
  Training_Accuracy = (len(y)-error_training)/len(y) * 100

  print('Training Accuracy:', Training_Accuracy)

  print('Total time in minutes:', Total_time/60)

  return a, bias, beta, Training_Accuracy

In [5]:
Lmax = 10000
input_nodes = 28**2
output_nodes = number_classes
a, bias, beta,Training_Accuracy = IELM_Train(input_nodes,Lmax,output_nodes,x_train,y_train)

Training Accuracy: 94.38499999999999
Total time in minutes: 17.238683887322743


In [6]:
# Testing
def IELM_Test(x,y,a,bias,beta):

  start_time = time.time()

  bias = bias
  p = np.dot(a,x.T) + bias
  H = sigmoid(p)
  Y_pred  = np.dot(H.T,beta)

  end_time = time.time()

  Total_time = end_time - start_time

  #Testing error
  error_test = 0

  for i in range(0,len(y)):
    desired_output = argmax(Y_pred[i,:])
    actual_output = argmax(y[i,:]) 

    if actual_output!=desired_output:
      error_test+=1

  # Testing Accuracy
  Testing_Accuracy = (len(y)-error_test)/len(y) * 100

  print('Testing Accuracy:', Testing_Accuracy)
  print('Total time in minutes:', Total_time/60)

  return Testing_Accuracy

In [7]:
a = a
bias = bias[:,:10000]
beta = beta
Training_Accuracy = IELM_Test(x_test,y_test,a,bias,beta)

Testing Accuracy: 94.38
Total time in minutes: 0.1285539666811625
